# Label side

In [1]:
import numpy as np
from jesse import helpers, research

_, candles = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "1m",
    helpers.date_to_timestamp("2022-07-01"),
    helpers.date_to_timestamp("2025-09-15"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)

candles.shape

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.12/site-packages/jesse/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


(1687680, 6)

In [2]:
np.save("data/btc_1m.npy", candles)
del candles

In [3]:
import numpy as np

candles = np.load("data/btc_1m.npy")
candles.shape

(1687680, 6)

In [4]:
from src.bars.fusion.deap_v1 import DeapBarV1

bar_container = DeapBarV1(max_bars=500000)
bar_container.update_with_candles(candles)
merged_bar = bar_container.get_fusion_bars()
merged_bar.shape

PyTorch configured: device=cpu, dtype=torch.float32


(5852, 6)

In [5]:
np.save("data/bar_deap_v1.npy", merged_bar)

In [6]:
from research.labeler.gmm_labeler import GMMLabeler

labeler_L5 = GMMLabeler(merged_bar, 5)
labeler_L5.plot_label_on_candles()

[I 2025-09-19 14:46:01,058] A new study created in memory with name: no-name-207a7128-47ca-4976-b067-facc807b6cd2
[I 2025-09-19 14:46:01,739] Trial 0 finished with value: 21.72226107045391 and parameters: {'random_state': 336}. Best is trial 0 with value: 21.72226107045391.
[I 2025-09-19 14:46:02,335] Trial 1 finished with value: 1.7779357885247815 and parameters: {'random_state': 196}. Best is trial 0 with value: 21.72226107045391.
[I 2025-09-19 14:46:02,876] Trial 2 finished with value: 21.146059112940975 and parameters: {'random_state': 38}. Best is trial 0 with value: 21.72226107045391.
[I 2025-09-19 14:46:03,468] Trial 3 finished with value: 1.7779357885247817 and parameters: {'random_state': 55}. Best is trial 0 with value: 21.72226107045391.
[I 2025-09-19 14:46:03,911] Trial 4 finished with value: 21.220052012393097 and parameters: {'random_state': 789}. Best is trial 0 with value: 21.72226107045391.
[I 2025-09-19 14:46:04,323] Trial 5 finished with value: 21.738446324767743 and

In [11]:
# labeler_L5.plot_label_returns()

In [7]:
labeler_L6 = GMMLabeler(merged_bar, 6)
labeler_L6.plot_label_on_candles()

[I 2025-09-19 14:46:07,465] A new study created in memory with name: no-name-60287e50-3ba8-4cf3-8a3c-c5b54bb182b9
[I 2025-09-19 14:46:07,972] Trial 0 finished with value: 17.38797821890597 and parameters: {'random_state': 292}. Best is trial 0 with value: 17.38797821890597.
[I 2025-09-19 14:46:08,509] Trial 1 finished with value: 1.7813783452803635 and parameters: {'random_state': 328}. Best is trial 0 with value: 17.38797821890597.
[I 2025-09-19 14:46:09,164] Trial 2 finished with value: 1.7813783452803633 and parameters: {'random_state': 835}. Best is trial 0 with value: 17.38797821890597.
[I 2025-09-19 14:46:09,667] Trial 3 finished with value: 22.22751608051619 and parameters: {'random_state': 698}. Best is trial 3 with value: 22.22751608051619.
[I 2025-09-19 14:46:10,366] Trial 4 finished with value: 22.24914195913021 and parameters: {'random_state': 881}. Best is trial 4 with value: 22.24914195913021.
[I 2025-09-19 14:46:10,837] Trial 5 finished with value: 17.400125011635847 and

In [8]:
labeler_L7 = GMMLabeler(merged_bar, 7)
labeler_L7.plot_label_on_candles()

[I 2025-09-19 14:46:14,211] A new study created in memory with name: no-name-1bc54954-0a4b-45ac-a1cc-9f1bf7b670d3
[I 2025-09-19 14:46:15,098] Trial 0 finished with value: 19.480890609804707 and parameters: {'random_state': 744}. Best is trial 0 with value: 19.480890609804707.
[I 2025-09-19 14:46:15,527] Trial 1 finished with value: 17.739595367771763 and parameters: {'random_state': 682}. Best is trial 0 with value: 19.480890609804707.
[I 2025-09-19 14:46:16,233] Trial 2 finished with value: 19.490535658544765 and parameters: {'random_state': 13}. Best is trial 2 with value: 19.490535658544765.
[I 2025-09-19 14:46:16,855] Trial 3 finished with value: 19.52205590529467 and parameters: {'random_state': 68}. Best is trial 3 with value: 19.52205590529467.
[I 2025-09-19 14:46:17,801] Trial 4 finished with value: 19.564245886246756 and parameters: {'random_state': 236}. Best is trial 4 with value: 19.564245886246756.
[I 2025-09-19 14:46:18,654] Trial 5 finished with value: 19.480890609804707

In [9]:
for labeler, idx in zip([labeler_L5, labeler_L6, labeler_L7], [5, 6, 7]):
    label_hard = labeler.label_hard_state
    print(f"{label_hard.shape = }")
    np.save(f"data/raw_label_hard_L{idx}.npy", label_hard)
    label_direction_force = labeler.label_direction_force
    print(f"{label_direction_force.shape = }")
    np.save(f"data/raw_label_direction_L{idx}.npy", label_direction_force)

label_hard.shape = (5847,)
label_direction_force.shape = (5847,)
label_hard.shape = (5846,)
label_direction_force.shape = (5846,)
label_hard.shape = (5845,)
label_direction_force.shape = (5845,)


# Label meta

In [1]:
import numpy as np
import pandas as pd
from strategies.BinanceBtcEntropyBarV1.config import SIDE
from strategies.BinanceBtcEntropyBarV1.config import get_side_model

side_model = get_side_model(False)

merged_bar = np.load("data/merged_bar.npy")

df_features = pd.read_parquet("data/features.parquet")

side_res = side_model.predict(df_features[SIDE])

side_pred_label = np.where(side_res > 0.5, 1, -1)
print(side_pred_label.shape)

print(np.unique(side_pred_label, return_counts=True))

(13219,)
(array([-1,  1]), array([5080, 8139]))


In [2]:
close_prices = merged_bar[:, 2]
len_gap = len(close_prices) - len(side_pred_label)
close_prices = close_prices[len_gap - 1 : -1]

assert len(close_prices) == len(side_pred_label)

In [3]:
meta_label = np.zeros(len(side_pred_label))

TRADE_FEE = 0.05 / 100

start_idx = 0
cumsum_ret = 0
start_price = 0
for idx, (i, p) in enumerate(zip(side_pred_label, close_prices)):
    if i == 1 or i == -1:
        if idx == 0:
            # 开始持仓
            start_idx = idx
            start_price = p
            cumsum_ret -= TRADE_FEE
        elif side_pred_label[idx - 1] != i:
            # 反向持仓，先结算收益
            cumsum_ret -= TRADE_FEE
            cumsum_ret += np.log(p / start_price) * side_pred_label[idx - 1]
            if cumsum_ret > 0:
                meta_label[start_idx:idx] = 1
            cumsum_ret = 0
            start_price = p
            start_idx = idx
            cumsum_ret -= TRADE_FEE
        else:
            # 继续持仓
            continue
    else:
        raise ValueError(f"side_pred_label[{idx}] = {i} is not valid")
else:
    last_price = merged_bar[-1, 2]
    # 结算最后一根bar的持仓, 可能还没有结算，所以先不加trade fee
    if i == side_pred_label[idx - 1]:
        # 已经开仓，结算
        cumsum_ret += np.log(last_price / start_price) * side_pred_label[idx - 1]
        if cumsum_ret > 0:
            meta_label[start_idx:] = 1
    else:
        # 反向开仓
        cumsum_ret -= TRADE_FEE
        cumsum_ret += np.log(last_price / start_price) * side_pred_label[idx - 1]
        if cumsum_ret > 0:
            meta_label[start_idx:] = 1


print(np.unique(meta_label, return_counts=True))

(array([0., 1.]), array([  669, 12550]))


In [4]:
np.save("data/label_meta.npy", meta_label)